# CSL PROJECT

In [51]:
# Import all packages needed
# pip install --upgrade py2neo
import py2neo
import pandas as pd

In [52]:
from py2neo import Graph

## Task 1

In [53]:
# define the uri, username and password for connecting to the toy database
uri_address = "neo4j+s://bbf4b768.databases.neo4j.io:7687"
username = "neo4j"
password = "2wlwMeE-FWEtYLXOkIoCK3f3900DpFBR5pLnQRLGK4I"

In [54]:
# Connect to the graph databse using uri authentication
graph = Graph(uri=uri_address, port = 7687, auth=(username, password))

## Task 2

In [6]:
# find all properties keys and values of nodes belonging to Process node type
properties_process = "MATCH (p:Process) RETURN COLLECT(p{.*})"
result_process =graph.evaluate(properties_process)
result_process

[{'Param5': 10,
  'Param4': 10,
  'Param3': 0.001,
  'Param2': 500,
  'ProcessID': 'P1',
  'Param1': 60},
 {'Param5': 10,
  'Param4': 10,
  'Param3': 0.001,
  'ProcessID': 'P2',
  'Param1': 130000000}]

In [7]:
# Create the dataframe for the results of properties
result_df1 = pd.DataFrame(result_process)
result_df1

,Param5,Param4,Param3,Param2,ProcessID,Param1
0,10,10,0.001,500.0,P1,60
1,10,10,0.001,NaN,P2,130000000


In [8]:
# find all properties keys and values of nodes belonging to Material node type
properties_material = "MATCH (m:Material) RETURN COLLECT(m{.*})"
result_material =graph.evaluate(properties_material)
result_material

[{'MaterialID': 'M1S', 'Owner': 'YOUR-NAME', 'Note': 'Start'},
 {'MaterialID': 'M1P', 'Owner': 'YOUR-NAME', 'Note': 'Positive'},
 {'MaterialID': 'M1N', 'Owner': 'YOUR-NAME', 'Note': 'Negative'},
 {'MaterialID': 'M2S', 'Owner': 'YOUR-NAME', 'Note': 'Start'},
 {'MaterialID': 'M2P', 'Owner': 'YOUR-NAME', 'Note': 'Positive'},
 {'MaterialID': 'M2N', 'Owner': 'YOUR-NAME', 'Note': 'Negative'}]

In [9]:
result_df2 = pd.DataFrame(result_material)
result_df2

,MaterialID,Owner,Note
0,M1S,YOUR-NAME,Start
1,M1P,YOUR-NAME,Positive
2,M1N,YOUR-NAME,Negative
3,M2S,YOUR-NAME,Start
4,M2P,YOUR-NAME,Positive
5,M2N,YOUR-NAME,Negative


In [10]:
print("The dataframe output for Process is:")
print(result_df1.to_string(index=False))
print("The dataframe output for Material is:")
print(result_df2.to_string(index=False))

The dataframe output for Process is:
 Param5  Param4  Param3  Param2 ProcessID    Param1
     10      10   0.001   500.0        P1        60
     10      10   0.001     NaN        P2 130000000
The dataframe output for Material is:
MaterialID     Owner     Note
       M1S YOUR-NAME    Start
       M1P YOUR-NAME Positive
       M1N YOUR-NAME Negative
       M2S YOUR-NAME    Start
       M2P YOUR-NAME Positive
       M2N YOUR-NAME Negative


# Task 3


In [11]:
# Test 1
# out_query = "MATCH (m:Material)-[:ProcessOutput]-(Process) WHERE m.MaterialID = 'M2P' RETURN m, collect(Process.ProcessID) AS ProcessID_From"
# test_result = graph.run(out_query)
# test_result

## Attempt Based on MaterialID

In [12]:
# material_ids = ['M1P', 'M1N', 'M2P', 'M2N']

# # Initialise an empty list to store the query results
# query_results = []

# # Loop through each MaterialID and run the query
# for material_id in material_ids:
#     query = (
#         f"MATCH (m:Material {{MaterialID: '{material_id}'}})-[rel:ProcessOutput|ProcessInput]-(p:Process)"
#         f"RETURN m.MaterialID AS MaterialID, m.Owner AS Owner, m.Note AS Note, "
#         f"CASE WHEN type(rel) = 'ProcessOutput' THEN p.ProcessID ELSE NaN END AS ProcessID_To, "
#         f"CASE WHEN type(rel) = 'ProcessInput' THEN p.ProcessID ELSE NaN END AS ProcessID_From"
#     )
#     result = graph.run(query).data()
#     query_results.extend(result)

# # Create a pandas DataFrame from the query results
# df = pd.DataFrame(query_results)

## Attempt2: Based on ProcessID

In [13]:
process_ids = ['P1','P2']

# Initialise an empty list to store the query results
query_results = []

# Loop through each MaterialID and run the query
for process_id in process_ids:
    query = (
        f"MATCH (p:Process {{ProcessID: '{process_id}'}})-[rel:ProcessOutput|ProcessInput]-(m:Material)"
        f"RETURN m.MaterialID AS MaterialID, m.Owner AS Owner, m.Note AS Note, "
        f"CASE WHEN type(rel) = 'ProcessInput' THEN p.ProcessID ELSE NaN END AS ProcessID_To, "
        f"CASE WHEN type(rel) = 'ProcessOutput' THEN p.ProcessID ELSE NaN END AS ProcessID_From"
    )
    result = graph.run(query).data()
    query_results.extend(result)

# Create a pandas DataFrame from the query results
df2 = pd.DataFrame(query_results)

In [14]:
df2

,MaterialID,Owner,Note,ProcessID_To,ProcessID_From
0,M1N,YOUR-NAME,Negative,NaN,P1
1,M1P,YOUR-NAME,Positive,NaN,P1
2,M1S,YOUR-NAME,Start,P1,NaN
3,M2N,YOUR-NAME,Negative,NaN,P2
4,M2P,YOUR-NAME,Positive,NaN,P2
5,M2S,YOUR-NAME,Start,P2,NaN


# Task 4

In [15]:
df2.columns

Index(['MaterialID', 'Owner', 'Note', 'ProcessID_To', 'ProcessID_From'], dtype='object')

In [60]:
# Create dataframes (Hardcode)
data_M = {
    'MaterialID' : ['M3S','M5S','M5P','M5N','M3P','M3N'],
    'Owner' : ['YUJIE'] *6,
    'Note' : ['Start']*2 +['Positive','Negative']*2,
    'ProcessID_To' : ['P3','P5'] + ['NaN']*4,
    'ProcessID_From' : ['NaN']*2 + ['P5']*2 + ['P3']*2
 }

data_P = {'ProcessID' : ['P5','P3'],
          'Param5' : ['10','10'],
          'Param4' : ['10','10'],
          'Param3' : ['0.001','0.001'],
          'Param2' : ['500','Nan'],
          'Param1' : ['60','130000000']
}
df_M = pd.DataFrame(data_M)
df_P = pd.DataFrame(data_P)

In [ ]:
# Create new Process Nodes
for _, row in df_P.iterrows():
    p1 = row["Param1"]
    p2 = row['Param2']
    p3 = row['Param3']
    p4 = row['Param4']
    p5 = row['Param5']
    pid = row['ProcessID']
    create_p_query = (
        f"CREATE (p:Process {{ Param1: $param1, Param2: $param2, Param3: $param3, Param4: $param4, Param5: $param5, ProcessID: $processid }})"
    )
    
    graph.run(create_p_query, param1 = p1, param2 = p2, param3 = p3, param4 = p4, param5 = p5, processid = pid)

## To check whether nodes are successfully created
# properties_process = "MATCH (p:Process) RETURN COLLECT(p{.*})"
# result_process =graph.evaluate(properties_process)
# result_process

In [61]:
# Create new Material Nodes and connect to Process nodes
for _, row in df_M.iterrows():
    mid = row['MaterialID']
    mowner = row['Owner']
    mnote = row['Note']
    to_id = row['ProcessID_To']
    from_id = row['ProcessID_From']
    # Create node
    create_m_query = (
        f"CREATE (m:Material {{ MaterialID: $materialid, Note: $note, Owner: $owner}})"
    )
    graph.run(create_m_query, materialid = mid, note = mnote, owner = mowner)
    
    # Create relationships
    if to_id != 'NaN':
        create_inrel_query = (
                "MATCH (m:Material { MaterialID: $materialid}), (p:Process { ProcessID: $processid}) "
                "CREATE (m)-[:ProcessInput]->(p)"
            )
        graph.run(create_inrel_query, materialid=mid, processid=to_id)


    if from_id != 'NaN':
        create_outrel_query = (
                "MATCH (m:Material { MaterialID: $materialid}), (p:Process { ProcessID: $processid}) "
                "CREATE (p)-[:ProcessOutput]->(m)"
            )
        graph.run(create_outrel_query, materialid=mid, processid=from_id)

In [59]:
delete_rel_query = (
    "MATCH (m:Material { MaterialID: $material_id })-[r:ProcessInput]->(p:Process { ProcessID: $process_id }) "
    "DELETE r"
)
graph.run(delete_rel_query, material_id= 'M3S', process_id='P3')

(No data)